### 0. 패키지 불러오기

In [1]:
import torch
import pandas as pd
import numpy as np
import time
import datetime
import gc

from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import BertModel
from transformers import BertModel, DistilBertModel
from transformers import BertTokenizer
from tokenizers import BertWordPieceTokenizer
from transformers import DistilBertTokenizer
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import DistilBertModel, DistilBertConfig
from transformers import get_linear_schedule_with_warmup

## warning 제거
import warnings
warnings.filterwarnings("ignore")

In [2]:
# GPU 여부 확인
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce RTX 2080 Ti


In [3]:
# 디바이스 GPU로 세팅하기 (가능하지 않을 시, CPU로 변경)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Using device: cuda

GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [4]:
torch.cuda.is_available()

True

### 1. 데이터 불러오기

In [5]:
df= pd.read_excel("C:/Users/User/Desktop/논문준비/news/뉴스 차종 라벨링_스택_v2.xlsx",names=['date', 'sentence','source', 'label','pos_neg', 'contents'])

In [6]:
# 문장 숫자 출력
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

Number of training sentences: 4,999



In [7]:
# 5개 샘플 로우 출력
df.sample(5)

,date,sentence,source,label,pos_neg,contents
4791,2020.08.31.,경제학자 10명 중 8명 “정부 정책으로 집값 폭등했다”,건설경제신문,0,0,한국경제학회 설문 결과…71%는 “임대차 3법으로 임차인 부담 더 커질 것” 국내 ...
114,2020.03.23.,내년부터 초소형 청소ㆍ세탁ㆍ소방차 나온다…초소형 특수차 차종 신설 추진,이투데이,4,1,특수차 차종 신설을 추진한다고 밝혔다. 현행 자동차 분류체계(자동차관리법 시행규칙)...
211,2020.02.27.,현대차·벤츠·BMW·아우디 등 5개사 26개 차종 1만2463대 리콜,머니S,3,2,26개 차종 1만2463대에서 제작결함이 발견돼 시정조치(리콜)한다고 27일 밝혔다...
337,2020.01.02.,현대자동차 2019년 총 442만 대 판매…가장 많이 팔린 차종은?,일요신문,1,1,차종별로는 투싼이 전세계 시장에서 67만 2141대의 판매고를 올리며 최다 판매 차...
2856,2019.12.30.,"테슬라, 中생산 전기차 `모델3` 구매자에게 첫 인도",매일경제,2,1,세계 최대 전기차시장서 첫 해외공장 가동 독일 베를린에 두번째 해외공장 추진중 미국...


In [8]:
dt=pd.DataFrame()

In [9]:
# 데이터 sentence, label 구분하기
sentences = (df['sentence'].astype(str)+df['contents'].astype(str)).values
#labels = df.label.values
labels = df.pos_neg.values
#pos_neg= df.pos_neg.values

In [10]:
## 문장 확인
sentences

array(["3세대 G80 출시 첫날 2만2000대 계약…현·기 차종 '역대 최다'이는 현대차와 기아차, 제네시스 모든 차종중 역대 최다 계약건수다. 또한 올 판매량 3만3000대의 3분의 2가 첫 날에 팔린 셈이다. 제네시스는 30일 온라인으로 출시행사를 갖고 G80의 완전 변경 모델 디 올 뉴 G80를...",
       '힘 센 놈, 날렵한 놈, 신기한 놈…베일 벗는 차차차대표하는 차종들이 새 얼굴로 바뀌었다. 하반기에도 신차 여러 종이 출시를 준비 중이다. 8세대 신형... 3세대 K5, 쏘나타 질주 따라잡을까 최근 10여년 동안 거셌던 스포츠실용차(SUV) 바람에 가장 큰 영향을 받은 차종...',
       "S60·V60·V90 등 볼보 전 차종 미국서 리콜…'브레이크 결함'볼보가 미국에서 최근 판매된 전 차종에 대해 리콜을 실시한다. 31일 미국도로교통안전국(NHTSA) 등에 따르면 볼보는 현지에서 지난해부터 최근까지 판매된 S60·V60·S90·V90·XC40·XC60·XC90 등 모든 차량에 대한 리콜을...",
       ...,
       '<별별 구독경제>음식물 쓰레기 고온건조 분쇄… 세균 걱정 ‘끝’김철회 모두렌탈 이사는 “최근 소비 트렌드가 구독 경제로 변화하면서 가전제품 렌털도 증가하고 있는... 이어 “에코체의 경우 제휴카드 사용 등을 통해 하루 564원 상당의 경제적인 금액에 음식물 쓰레기...',
       '한은, 거시계량모형 BOK20 구축…"2012년 이후 경제상황 반영"한국은행은 2012년 이후 우리나라의 경제구조 변화를 반영하고 거시경제를 보다 현실적으로 분석하기 위해 기존의 \'거시계량모형\'을 보완한 BOK20 모형을 구축했다고 30일 밝혔다. 앞서 한은은 경제충격에 대한...',
       "파주시,'조선 최대의 실용백과사전 임원경제지' 학술대회 개최▲ 파주시,'조선 최대의 실용백과사전 임원경제지' 학술대회 개최 [경기 위클리오늘] 파주시는 오는 9월 3일 '조선 최대의 실용백과사전 임원경제지' 학술

In [11]:
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import BertModel
from transformers import BertModel, DistilBertModel
#from tokenization_kobert import KoBertTokenizer
from transformers import BertTokenizer
from tokenizers import BertWordPieceTokenizer
from transformers import DistilBertTokenizer

tokenizer = BertTokenizer.from_pretrained("beomi/kcbert-base")

# Load the BERT tokenizer.
#print('Loading BERT tokenizer...')
#tokenizer = BertTokenizer.from_pretrained('albert-base-v2', do_lower_case=True)

#model = BertModel.from_pretrained('monologg/kobert')
#tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
#tokenizer = AutoTokenizer.from_pretrained(path)

## Base Model (108M)
#tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
#model = AutoModelWithLMHead.from_pretrained("beomi/kcbert-base")

# Large Model (334M)
#model = AutoModelWithLMHead.from_pretrained("beomi/kcbert-base")


#bert_model = BertModel.from_pretrained('monologg/kobert')

In [12]:
# Print the original sentence.
print(' Original: ', sentences[0])

print("")

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences[0]))

print("")

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

 Original:  3세대 G80 출시 첫날 2만2000대 계약…현·기 차종 '역대 최다'이는 현대차와 기아차, 제네시스 모든 차종중 역대 최다 계약건수다. 또한 올 판매량 3만3000대의 3분의 2가 첫 날에 팔린 셈이다. 제네시스는 30일 온라인으로 출시행사를 갖고 G80의 완전 변경 모델 디 올 뉴 G80를...

Tokenized:  ['3', '##세대', 'G', '##80', '출시', '첫', '##날', '2만', '##200', '##0', '##대', '계약', '[UNK]', '현', '·', '기', '차', '##종', "'", '역대', '최', '##다', "'", '이는', '현대차', '##와', '기아', '##차', ',', '제', '##네', '##시스', '모든', '차', '##종', '##중', '역대', '최', '##다', '계약', '##건', '##수다', '.', '또한', '올', '판매', '##량', '3만', '##30', '##00', '##대의', '3', '##분의', '2', '##가', '첫', '날', '##에', '팔', '##린', '셈', '##이다', '.', '제', '##네', '##시스', '##는', '30', '##일', '온라인으로', '출시', '##행사', '##를', '갖고', 'G', '##80', '##의', '완전', '변경', '모델', '디', '올', '뉴', 'G', '##80', '##를', '.', '.', '.']

Token IDs:  [22, 11372, 40, 16212, 22693, 2881, 4272, 26306, 28490, 4263, 4140, 12446, 1, 3432, 97, 414, 2839, 4197, 10, 9144, 2925, 4020, 10, 15779, 18220, 4196, 27366, 4495, 15, 2545, 4011, 27357, 8229, 2839, 4197, 4276, 9144, 2925, 4020, 1244

In [13]:
max_len = 0

# For every sentence...
for sent in sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  140


In [14]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_len,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

...

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Original:  3세대 G80 출시 첫날 2만2000대 계약…현·기 차종 '역대 최다'이는 현대차와 기아차, 제네시스 모든 차종중 역대 최다 계약건수다. 또한 올 판매량 3만3000대의 3분의 2가 첫 날에 팔린 셈이다. 제네시스는 30일 온라인으로 출시행사를 갖고 G80의 완전 변경 모델 디 올 뉴 G80를...
Token IDs: tensor([    2,    22, 11372,    40, 16212, 22693,  2881,  4272, 26306, 28490,
         4263,  4140, 12446,     1,  3432,    97,   414,  2839,  4197,    10,
         9144,  2925,  4020,    10, 15779, 18220,  4196, 27366,  4495,    15,
         2545,  4011, 27357,  8229,  2839,  4197,  4276,  9144,  2925,  4020,
        12446,  4164, 21216,    17, 10384,  2303, 11341,  4667, 21419, 11243,
         8040, 10772,    22, 14311,    21,  4009,  2881,   595,  4113,  3236,
         4331,  1869,  7976,    17,  2545,  4011, 27357,  4008,  8476,  4046,
        28672, 22693, 15583,  4180, 10206,    40, 16212,  4042,  8357, 20548,
        16505,   974,  2303,   743,    40, 16212,  4180,    17,    17,    17,
            3,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,  

In [15]:
from torch.utils.data import TensorDataset, random_split
import torch

torch.manual_seed(42)

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

4,499 training samples
  500 validation samples


In [16]:
train_dataset

In [17]:
val_dataset

In [18]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [19]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import DistilBertModel, DistilBertConfig
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

i=7
path= 'C:/Users/User/Desktop/논문준비/news/kcbert_distillation/distillation/serialization_dir/my_first_training'+str(i)+'/'


#config = DistilBertConfig.from_pretrained('C:/Users/User/Desktop/논문준비/news/DistilKoBERT-master/distillation/training_configs/distilbert-base-cased/distilkobert_6_layer.json', output_hidden_states=True)
#model = BertForSequenceClassification.from_pretrained(
#    'beomi/kcbert-base', # Use the 12-layer BERT model, with an uncased vocab.
#    num_labels = len(df['label'].unique()), # The number of output labels--2 for binary classification.
#                    # You can increase this for multi-class tasks.   
#    output_attentions = False, # Whether the model returns attentions weights.
#    output_hidden_states = False # Whether the model returns all hidden-states.
#)

## 사용 가능한 pre-trained 모델들
##monologg/koelectra-small-discriminator
#monologg/distilkobert
#monologg/koelectra-base-discriminator
#monologg/kobert
#beomi/kcbert-base
#albert-base-v2

model = DistilBertForSequenceClassification.from_pretrained(
    path, # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = len(df['label'].unique()),
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False # Whether the model returns all hidden-states.
)

model.cuda()

Some weights of the model checkpoint at C:/Users/User/Desktop/논문준비/news/kcbert_distillation/distillation/serialization_dir/my_first_training7/ were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at C:/Users/User/Desktop/논문준비/n

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [20]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 104 different named parameters.

==== Embedding Layer ====

distilbert.embeddings.word_embeddings.weight            (30000, 768)
distilbert.embeddings.position_embeddings.weight          (300, 768)
distilbert.embeddings.LayerNorm.weight                        (768,)
distilbert.embeddings.LayerNorm.bias                          (768,)
distilbert.transformer.layer.0.attention.q_lin.weight     (768, 768)

==== First Transformer ====

distilbert.transformer.layer.0.attention.q_lin.bias           (768,)
distilbert.transformer.layer.0.attention.k_lin.weight     (768, 768)
distilbert.transformer.layer.0.attention.k_lin.bias           (768,)
distilbert.transformer.layer.0.attention.v_lin.weight     (768, 768)
distilbert.transformer.layer.0.attention.v_lin.bias           (768,)
distilbert.transformer.layer.0.attention.out_lin.weight   (768, 768)
distilbert.transformer.layer.0.attention.out_lin.bias         (768,)
distilbert.transformer.layer.0.sa_layer_norm.weight           (

In [21]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 3e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [22]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 30

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [23]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [24]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [25]:
import gc
gc.collect()

5438

In [26]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids, 
        #                     token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.3f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                   #token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.3f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.3f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 30 ========
Training...
  Batch    40  of    141.    Elapsed: 0:00:06.
  Batch    80  of    141.    Elapsed: 0:00:11.
  Batch   120  of    141.    Elapsed: 0:00:16.

  Average training loss: 0.788
  Training epcoh took: 0:00:18

Running Validation...
  Accuracy: 0.757
  Validation Loss: 0.546
  Validation took: 0:00:00

======== Epoch 2 / 30 ========
Training...
  Batch    40  of    141.    Elapsed: 0:00:05.
  Batch    80  of    141.    Elapsed: 0:00:10.
  Batch   120  of    141.    Elapsed: 0:00:15.

  Average training loss: 0.579
  Training epcoh took: 0:00:18

Running Validation...
  Accuracy: 0.817
  Validation Loss: 0.518
  Validation took: 0:00:00

======== Epoch 3 / 30 ========
Training...
  Batch    40  of    141.    Elapsed: 0:00:05.
  Batch    80  of    141.    Elapsed: 0:00:10.
  Batch   120  of    141.    Elapsed: 0:00:15.

  Average training loss: 0.487
  Training epcoh took: 0:00:18

Running Validation...
  Accuracy: 0.864
  Validation Loss: 0.369
  Va

  Batch    40  of    141.    Elapsed: 0:00:05.
  Batch    80  of    141.    Elapsed: 0:00:10.
  Batch   120  of    141.    Elapsed: 0:00:15.

  Average training loss: 0.143
  Training epcoh took: 0:00:18

Running Validation...
  Accuracy: 0.922
  Validation Loss: 0.231
  Validation took: 0:00:00

======== Epoch 26 / 30 ========
Training...
  Batch    40  of    141.    Elapsed: 0:00:05.
  Batch    80  of    141.    Elapsed: 0:00:10.
  Batch   120  of    141.    Elapsed: 0:00:15.

  Average training loss: 0.149
  Training epcoh took: 0:00:18

Running Validation...
  Accuracy: 0.922
  Validation Loss: 0.216
  Validation took: 0:00:00

======== Epoch 27 / 30 ========
Training...
  Batch    40  of    141.    Elapsed: 0:00:05.
  Batch    80  of    141.    Elapsed: 0:00:10.
  Batch   120  of    141.    Elapsed: 0:00:15.

  Average training loss: 0.146
  Training epcoh took: 0:00:18

Running Validation...
  Accuracy: 0.928
  Validation Loss: 0.219
  Validation took: 0:00:00

======== Epoch 28 

In [27]:
#dt=pd.DataFrame()

In [28]:
total_sec= time.time()-total_t0

In [29]:
test_dt= pd.DataFrame(training_stats)

In [30]:
test_dt['total_seconds']=total_sec
test_dt['model_index']=i

In [31]:
dt= pd.concat([test_dt,dt])

In [32]:
dt

,Training Loss,Training Time,Valid. Accur.,Valid. Loss,Validation Time,epoch,total_seconds,model_index
0,0.788271,0:00:18,0.757422,0.545591,0:00:00,1,554.930821,7
1,0.578729,0:00:18,0.816797,0.518345,0:00:00,2,554.930821,7
2,0.487330,0:00:18,0.863672,0.368649,0:00:00,3,554.930821,7
3,0.425549,0:00:18,0.863672,0.348329,0:00:00,4,554.930821,7
4,0.393150,0:00:18,0.877734,0.298377,0:00:00,5,554.930821,7
5,0.389518,0:00:18,0.887500,0.287647,0:00:00,6,554.930821,7
6,0.347941,0:00:18,0.904297,0.242765,0:00:00,7,554.930821,7
7,0.316559,0:00:18,0.895312,0.249632,0:00:00,8,554.930821,7
8,0.292740,0:00:18,0.908203,0.236275,0:00:00,9,554.930821,7
9,0.268067,0:00:18,0.904297,0.246484,0:00:00,10,554.930821,7


In [33]:
#dt.to_csv("C:/Users/User/Desktop/논문준비/news/kcbert_distillation/distillation/test.txt")

In [34]:
logits

array([[ 6.4239044e+00,  8.5351837e-01,  9.3342364e-04, -5.6081223e+00,
        -5.8976336e+00, -5.7768493e+00, -5.0867906e+00],
       [ 8.3502617e+00, -1.5250114e+00, -2.3230710e+00, -5.2294121e+00,
        -5.9084182e+00, -5.6137285e+00, -4.4340315e+00],
       [ 8.2881241e+00, -3.1981959e+00, -1.9570218e+00, -4.9790182e+00,
        -5.8706465e+00, -5.2190146e+00, -4.1099052e+00],
       [ 5.3570623e+00,  7.8032494e-01,  1.6087167e+00, -5.6315155e+00,
        -5.8747473e+00, -5.6309767e+00, -5.1362820e+00],
       [ 1.7530601e+00,  4.8753290e+00,  1.3196332e+00, -5.2343082e+00,
        -4.7392244e+00, -5.1773520e+00, -4.9292722e+00],
       [ 1.0341617e+00,  5.9760828e+00,  1.5534563e-01, -4.7591200e+00,
        -4.3333330e+00, -4.9647269e+00, -4.3243346e+00],
       [-1.2247555e-02,  5.1097617e+00,  2.1243024e+00, -5.1252117e+00,
        -4.8171549e+00, -5.0733385e+00, -4.9123583e+00],
       [ 8.3608541e+00, -1.7042993e+00, -2.2929723e+00, -5.1767073e+00,
        -5.9072866e+00, -

In [35]:
label_ids

array([0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 2, 0, 1, 0, 0],
      dtype=int64)

In [36]:
loss

tensor(0.0102, device='cuda:0')